In [14]:
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel, Field
import torch
from transformers import BertTokenizerFast, BertForTokenClassification, BertForSequenceClassification
import nest_asyncio
import os
import spacy

nest_asyncio.apply()

app = FastAPI()

class Item(BaseModel):
    text: str = Field(..., example="""Fiber 100mb SuperOnline kullanıcısıyım yaklaşık 2 haftadır @Twitch @Kick_Turkey gibi canlı yayın platformlarında 360p yayın izlerken donmalar yaşıyoruz. Başka hiç bir operatörler bu sorunu yaşamazken ben parasını verip alamadığım hizmeti neden ödeyeyim ? @Turkcell """)

# Dosya yollarını kontrol etme
ner_model_path = "fine-tuned-bert-ner-optimized"
sentiment_model_path = "entity_sentiment_model"
sentiment_tokenizer_path = "entity_sentiment_tokenizer"

if not os.path.exists(ner_model_path):
    raise FileNotFoundError(f"NER modeli dizini bulunamadı: {ner_model_path}")
if not os.path.exists(sentiment_model_path):
    raise FileNotFoundError(f"Sentiment modeli dizini bulunamadı: {sentiment_model_path}")
if not os.path.exists(sentiment_tokenizer_path):
    raise FileNotFoundError(f"Sentiment tokenizer dizini bulunamadı: {sentiment_tokenizer_path}")

# NER modelini yükleme
tokenizer_ner = BertTokenizerFast.from_pretrained(ner_model_path)
model_ner = BertForTokenClassification.from_pretrained(ner_model_path)
model_ner.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Sentiment modelini yükleme
tokenizer_sentiment = BertTokenizerFast.from_pretrained(sentiment_tokenizer_path)
model_sentiment = BertForSequenceClassification.from_pretrained(sentiment_model_path)
model_sentiment.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Etiketler
id_to_label_ner = {0: "O", 1: "B-ORG", 2: "I-ORG"}
id_to_label_sentiment = {0: "notr", 1: "olumlu", 2: "olumsuz"}

# Spacy modeli yükleme
nlp = spacy.blank("tr")
nlp.add_pipe('sentencizer')

def extract_entities(text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = tokenizer_ner(text.lower(), truncation=True, padding='max_length', max_length=512, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    model_ner.eval()
    with torch.no_grad():
        outputs = model_ner(**inputs)
        predictions = torch.argmax(outputs.logits, dim=2)
    tokens = tokenizer_ner.convert_ids_to_tokens(inputs['input_ids'][0])
    predicted_labels = [id_to_label_ner[pred.item()] for pred in predictions[0]]
    entities = []
    current_entity = []
    for token, label in zip(tokens, predicted_labels):
        if token.startswith("##"):
            if current_entity:
                current_entity[-1] += token[2:]
        else:
            if label.startswith("B-"):
                if current_entity:
                    entities.append(" ".join(current_entity))
                    current_entity = []
                if token != "[PAD]":
                    current_entity.append(token)
            elif label.startswith("I-") and current_entity:
                if token != "[PAD]":
                    current_entity.append(token)
            else:
                if current_entity:
                    entities.append(" ".join(current_entity))
                    current_entity = []
    if current_entity:
        entities.append(" ".join(current_entity))
    entities = [entity for entity in entities if entity.strip() and entity.lower() != '[pad]']
    
    # Orijinal metindeki entity'leri bulma
    original_entities = []
    for entity in entities:
        start_idx = text.lower().find(entity.lower())
        end_idx = start_idx + len(entity)
        original_entity = text[start_idx:end_idx]
        original_entities.append(original_entity)
    
    return original_entities

def split_sentences(text, entities):
    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents]
    entity_sentences = {entity: [] for entity in entities}
    for sentence in sentences:
        for entity in entities:
            if entity.lower() in sentence.lower():
                entity_sentences[entity].append(sentence)
    return entity_sentences

def analyze_sentiment(text, entities):
    results = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    entity_sentences = split_sentences(text, entities)
    for entity, sentences in entity_sentences.items():
        combined_sentence = " ".join(sentences)
        inputs = tokenizer_sentiment(combined_sentence, truncation=True, padding='max_length', max_length=512, return_tensors="pt")
        inputs = {k: v.to(device) for k, v in inputs.items()}
        model_sentiment.eval()
        with torch.no_grad():
            outputs = model_sentiment(**inputs)
            predictions = torch.argmax(outputs.logits, dim=1)
        predicted_sentiment = id_to_label_sentiment[predictions.item()]
        results.append({"entity": entity, "sentiment": predicted_sentiment})
    return results

@app.post("/predict/", response_model=dict)
async def predict(item: Item):
    text = item.text
    entities = extract_entities(text)
    sentiment_results = analyze_sentiment(text, entities)
    result = {
        "entity_list": entities,
        "results": sentiment_results
    }
    return result

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=7552)


Task exception was never retrieved
future: <Task finished name='Task-73' coro=<Server.serve() done, defined at c:\Users\muham\AppData\Local\Programs\Python\Python312\Lib\site-packages\uvicorn\server.py:67> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "c:\Users\muham\AppData\Local\Programs\Python\Python312\Lib\site-packages\uvicorn\main.py", line 577, in run
    server.run()
  File "c:\Users\muham\AppData\Local\Programs\Python\Python312\Lib\site-packages\uvicorn\server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\muham\AppData\Roaming\Python\Python312\site-packages\nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\muham\AppData\Roaming\Python\Python312\site-packages\nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "C:\Users\muham\AppData\Roaming\Python\Py

INFO:     127.0.0.1:51985 - "POST /predict/ HTTP/1.1" 200 OK
